In [1]:

import os, sys
if 'Linux' in os.uname():
    from google.colab import drive
    drive.mount('/ME')
    predir='/ME/My Drive/'
else:
    predir='/Users/amit/Google Drive/'
datadirs=predir+'Colab Notebooks/FA/'
datadirsa=predir+'Colab Notebooks/STVAE/_CODE'

sys.path.insert(1, datadirs)
sys.path.insert(1, datadirsa)
savepath = datadirs+'save/'
datapath = predir+'LSDA_data/'
!pip install ninja
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from pprint import pprint
from mprep import get_data_pre
# layers
from layers import FALinear, FAConv2d
# Module classes and training and testing routines
from utils import *
from layerwise_networks import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
dtype = torch.float32

Drive already mounted at /ME; to attempt to forcibly remount, call drive.mount("/ME", force_remount=True).
Drive already mounted at /ME; to attempt to forcibly remount, call drive.mount("/ME", force_remount=True).
Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/cudnn_convolution/build.ninja...
Building extension module cudnn_convolution...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module cudnn_convolution...
Drive already mounted at /ME; to attempt to forcibly remount, call drive.mount("/ME", force_remount=True).
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module cudnn_convolution, skipping build step...
Loading extension module cudnn_convolution...
cuda:0


In [2]:
class pars:
    def __init__(self):

      self.OPT='SGD'
      self.layerwise=True
      self.device=device
      self.fa=0
      self.START_LAYER = 0
      self.NUM_LAYERS = 4
      self.NUM_CHANNEL = 32 
      self.HW = 32
      self.epochs = 2
      self.batch_size= 500
      self.MX=1.
      self.LR=[.1,.1,.1,.1,.1]
      self.CR='CE'
      self.get_net=get_net_b
      self.nval=5000
      self.num_train=50000-self.nval
      self.cl=None
      self.num_test=0
      self.edges=False
      self.mb_size=self.batch_size
      self.dataset='cifar10'
      self.num_class=10

pars=pars()
class layer_pars:
    def __init__(self,pars):
      self.NCOLD=3
      self.NC=pars.NUM_CHANNEL
      self.HW=pars.HW
pars.layer_pars=layer_pars(pars)


In [3]:

data =get_data_pre(pars,pars.dataset)
data=[data[0][0], data[0][1], data[1][0], data[1][1], data[2][0], data[2][1]]
#dataa = get_cifar10_man(datapath, num_train=45000)

/ME/My Drive/LSDA_data/CIFAR/cifar10_train.hdf5
tr (50000, 32, 32, 3)
n_classes 2.0 dim 32 nchannels 3
In get_data_pre: num_train 45000
Num test: 10000


In [ ]:
pars.fout=sys.stdout #open('OUT_'+pars.get_net.__name__+'.txt','w')
#logi=[[False, 0], [False, 1], [False, 2]]
logi=[[True, 0]]

cori=['CE','H']

for ll in logi:
  for cc in cori:
    pars.layerwise=ll[0]
    pars.fa=ll[1]
    pars.CR=cc
    pars.layer_pars=layer_pars(pars)
    get_net_pre(pars)
    LW=run_net(data,pars,savepath)
    show_results(pars, savepath, LW)
if pars.fout!=sys.stdout:
  pars.fout.close()

In [13]:
def train_mod(data, fix, model, pars, ep_loss, ep_acc):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    device=pars.device
    dtype = torch.float32
    train_dat=data[0]; train_tar=data[1]
    val_dat=data[2]; val_tar=data[3]
    if pars.layerwise:
       fix = fix.to(device=device)
       model=model.to(device=device)
    else:
      model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(pars.epochs):
        t1=time.time()
        for j in np.arange(0,len(train_tar),pars.batch_size):
            t2=time.time()

            model.train()  # put model to training mode
            x = torch.from_numpy(train_dat[j:j+pars.batch_size]).to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = torch.from_numpy(train_tar[j:j+pars.batch_size]).to(device=device, dtype=torch.long)
            t3=time.time()
            print('dat', t3- t2)
            if pars.layerwise:
              with torch.no_grad():
                   x1 = fix(x)
              scores = model(x1)
            else:
              x1=x
              scores,_ = model.forward(x1)
            
            loss = pars.criterion(scores, y)
            t4=time.time()
            print('for',t4-t3)
            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            pars.optimizer.zero_grad()
            #t2 = time.time()
            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            pars.optimizer.step()
            print('bak',time.time()-t4)


        #acc = check_accuracy(val_dat, val_tar, fix, model, pars)
        acc=0
        print('Epoch %d, loss = %.4f, val.acc = %.4f, time=%.2f\n' % (e, loss.item(), acc, time.time()-t1), file=pars.fout)

        ep_loss.append(loss.item())
        ep_acc.append(acc)
        pars.fout.flush()


In [5]:
lw_loss = []
lw_acc = []
lw_test_acc = []
fix = nn.Sequential()
pars.i = 0
pars.fout=sys.stdout
pars.layer_pars=layer_pars(pars)
pars.layerwise=True
get_net_pre(pars)
net, fix, layer = pars.get_net(pars, fix, pars.layer_pars)
print(net)


FACONV2D cuda
Sequential(
  (layer): Sequential(
    (0): FAConv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Hardtanh(min_val=0, max_val=1.0)
    (2): Dropout(p=0.5, inplace=False)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (aux): Sequential(
    (0): AvgPool2d(kernel_size=8, stride=8, padding=0)
    (1): Flatten()
    (2): FALinear(in_features=128, out_features=10, bias=True)
  )
)


In [13]:
print(device)

cuda


In [11]:
train_mod(data, fix, net, pars, ep_loss=lw_loss, ep_acc=lw_acc)


NameError: ignored

In [8]:
import torch.optim as optim
lw_loss = []
lw_acc = []
lw_test_acc = []
fix=None
pars.fout=sys.stdout
pars.criterion=torch.nn.CrossEntropyLoss()
pars.optimizer = optim.SGD(model.parameters(), lr=.1)
pars.layerwise=False
print(model)

network(
  (HT): Hardtanh(min_val=0.0, max_val=1.0)
  (ed): Edge()
  (criterion): CrossEntropyLoss()
  (criterion_shift): CrossEntropyLoss()
  (layers): ModuleList(
    (conv1): FAConv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (drop1): Dropout(p=0.5, inplace=False)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=[0], dilation=1, ceil_mode=False)
    (Avg): AvgPool2d(kernel_size=(8, 8), stride=(8, 8), padding=0)
    (Flat): Flatten()
    (dense_final): FALinear(in_features=128, out_features=10, bias=True)
  )
)


In [12]:

train_mod(data, fix, model, pars, ep_loss=lw_loss, ep_acc=lw_acc)


dat 0.0019593238830566406


TypeError: ignored

In [4]:
import tex
#
#importlib.reload(tex)
tex.make_tex('save/')

In [5]:
from aux_colab import seq, copy_to_content, copy_from_content, save_net, train_net, run_net
from model_layers import FALinear, FAConv2d

import mprep
import argparse
import aux
from get_net_text import get_network
import pprint
import network
def main_loc(par_file, device):

  if 'Linux' in os.uname():
        predir = '/ME/My Drive/'
  else:
        predir = '/Users/amit/Google Drive/'

  datadirs = predir + 'Colab Notebooks/STVAE/'
  model_out=None
  parser = argparse.ArgumentParser(fromfile_prefix_chars='@',
      description='Variational Autoencoder with Spatial Transformation')

  parser=aux.process_args(parser)
  f=open(par_file+'.txt','r')
  args=parser.parse_args(f.read().split())
  f.close()
  args.datadirs=datadirs

  # reinit means you are taking part of an existing network as fixed and updating some other parts.
  if args.rerun or args.reinit:
      args.run_existing=True

  ARGS, STRINGS, EX_FILES, SMS = mprep.get_names(args)
  # Get data device and output file
  fout= mprep.setups(args, EX_FILES)
  args.fout=fout
  # Get data
  #DATA=mprep.get_data_pre(args,args.dataset)

  
  sh=data[0].shape
  # parse the existing network coded in ARGS[0]
  arg=ARGS[0]
  
  arg.lnti, arg.layers_dict = get_network(arg.layers,nf=sh[1])
          # Initialize the network
  model = network.network(device, arg, arg.layers_dict, arg.lnti, fout, sh)

  return model, arg

In [6]:
copy_to_content('try',predir)
model, args=main_loc('try',device)

input [ 1  3 32 32]
conv1 [ 1 32 32 32]
drop1 [ 1 32 32 32]
pool1 [ 1 32 16 16]
Avg [ 1 32  2  2]
dense_final [ 1 10]
layers.conv1.weight,[32  3  5  5]
layers.conv1.bias,[32]
layers.conv1.weight_fb,[32  3  5  5]
layers.dense_final.weight,[ 10 128]
layers.dense_final.weight_fb,[ 10 128]
layers.dense_final.bias,[10]
tot_pars,7402
TO optimizer layers.conv1.weight[32  3  5  5]
TO optimizer layers.conv1.bias[32]
TO optimizer layers.conv1.weight_fb[32  3  5  5]
TO optimizer layers.dense_final.weight[ 10 128]
TO optimizer layers.dense_final.weight_fb[ 10 128]
TO optimizer layers.dense_final.bias[10]
Optimizer SGD 0.05

In [7]:

#from models_make import train_model

#train_model(model,args,'bla',data,sys.stdout)
import time
for i in range(3):
    time1=time.time()
    model.to(device)
    model.run_epoch([data[0],data[0],data[1]],0,0,None,None,None,d_type='train',fout=sys.stdout)
    print(time.time()-time1)

RuntimeError: ignored

In [ ]:
full_loss=0; full_acc=0;
# Loop over batches.
jump=500
epoch=0
fout=sys.stdout
d_type="train"
DD=data[0]
TT=data[1]
num_tr=data[0].shape[0]

for e in range(5):
  t0 = time.time()
  for j in np.arange(0, num_tr, jump,dtype=np.int32):
    t3=time.time()
    model.train()
    dat = torch.from_numpy(DD[j:j + jump]).to(device,dtype=torch.float32)
    target = torch.from_numpy(TT[j:j + jump]).to(device, dtype=torch.long)

    t1=time.time()
    print('dat',t1-t3)
    #with torch.no_grad() if (d_type!='train') else dummy_context_mgr():
    loss, acc= model.loss_and_grad(dat, target, 'train')


    full_loss += loss #.item()
    full_acc += acc #.item()
    #print('batch', time.time() - t3)
  if (True): #np.mod(epoch,10)==9 or epoch<=10):
    full_loss=np.float32(full_loss.detach().cpu().numpy())
    full_acc=np.float32(full_acc.detach().cpu().numpy())

    fout.write('\n ====> Ep {}: {} Full loss: {:.4F}, Full acc: {:.4F} \n'.format(d_type,epoch,
            full_loss /(num_tr/jump), full_acc/(num_tr)))
    
  print(time.time()-t0)

In [ ]:
copy_to_content('try',predir)
run_net('try')